In [ ]:
!pip install pandas numpy matplotlib seaborn scikit-learn plotly networkx statsmodels rpy2 shap streamlit streamlit_lottie fastapi scikit-bio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 27.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.7/11.7 MB 86.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for scikit-bio: filename=scikit_bio-0.6.2-cp310-cp310-linux_x86_64.whl size=3971913 sha256=9d74b0273894d48ac456228c12f899c63c34c5fc85c3f3a546f96787cf2c5771
  Stored in directory: /root/.cache/pip/wheels/f3/9c/ed/c2b94011fd3765a19330df93f81265e178098dd86c047c1566
  Created wheel for biom-format: filename=biom_format-2.1.16-cp310-cp310-linux_x86_64.whl size=12158967 sha256=3630f6e823df8c260956ecd88cd878a96764ca5fd027a8ebe4fac78ae620e574
  Stored in directory: /root/.cache/pip/wheels/8e/a9/f9/197fd5a0e5bbab5f2e03c89194f6c194bed7af5d7a8c8759f3
Successf

In [ ]:
%%writefile app.py
import streamlit as st
import os
import time
from time_series_derivatives_of_microbiome_dynamics_stlit_app import MicrobiomeAnalysisPipeline
import base64
from streamlit_lottie import st_lottie
import json

# Function to load Lottie JSON
def load_lottie_json(file_path):
    with open(file_path, "r") as file:
        return json.load(file)

# Function to display plots
def display_plot(file_path, title):
    with open(file_path, "rb") as file:
        base64_image = base64.b64encode(file.read()).decode()
    st.markdown(
        f"""
        <div class="plot-card" onclick="this.classList.toggle('expanded')">
            <h3>{title}</h3>
            <img src="data:image/png;base64,{base64_image}" alt="{title}">
        </div>
        """,
        unsafe_allow_html=True
    )

# Set page config
st.set_page_config(page_title="Microbiome Analysis", layout="wide")

# Custom CSS for dark/light mode and animations
st.markdown("""
<style>
@media (prefers-color-scheme: dark) {
    .stApp {
        background-color: #1E1E1E;
        color: #FFFFFF;
    }
}
@media (prefers-color-scheme: light) {
    .stApp {
        background-color: #FFFFFF;
        color: #000000;
    }
}
.plot-card {
    border: 1px solid #ddd;
    border-radius: 8px;
    padding: 10px;
    margin-bottom: 20px;
    transition: all 0.3s ease;
    cursor: pointer;
}
.plot-card img {
    max-width: 100%;
    transition: all 0.3s ease;
}
.plot-card.expanded {
    position: fixed;
    top: 50%;
    left: 50%;
    transform: translate(-50%, -50%);
    z-index: 1000;
    width: 90vw;
    height: 90vh;
    background-color: rgba(0, 0, 0, 0.8);
    display: flex;
    flex-direction: column;
    justify-content: center;
    align-items: center;
}
.plot-card.expanded img {
    max-width: 90%;
    max-height: 80vh;
}
</style>
""", unsafe_allow_html=True)

# App title
st.title("Microbiome Analysis Pipeline")

# File upload
otu_file = st.file_uploader("Upload OTU table (CSV)", type="csv")
metadata_file = st.file_uploader("Upload metadata file (CSV)", type="csv")

if otu_file and metadata_file:
    if st.button("Run Analysis"):
        # Save uploaded files
        otu_path = f"temp_otu_{time.time()}.csv"
        metadata_path = f"temp_metadata_{time.time()}.csv"
        with open(otu_path, "wb") as f:
            f.write(otu_file.getbuffer())
        with open(metadata_path, "wb") as f:
            f.write(metadata_file.getbuffer())

        # Create progress bar
        progress_bar = st.progress(0)

        # Load and display processing animation
        lottie_processing = load_lottie_json("/content/lottie.json")
        lottie_placeholder = st_lottie(lottie_processing, speed=1, height=200, key="processing")

        # Run analysis
        pipeline = MicrobiomeAnalysisPipeline(otu_path, metadata_path)
        files = pipeline.run_and_get_analyses()

        # Update progress and hide animation
        progress_bar.progress(100)
        lottie_placeholder.empty()

        # Display plots
        st.header("Analysis Results")
        for file in files:
            title = file.split('/')[-1].split('.')[0].replace('_', ' ').title()
            display_plot(file, title)

        # Clean up temporary files
        os.remove(otu_path)
        os.remove(metadata_path)

# Add some information about the app
st.sidebar.header("About")
st.sidebar.info("This app performs microbiome analysis based on uploaded OTU table and metadata files.")
st.sidebar.info("Upload your files and click 'Run Analysis' to start.")

# JavaScript for handling plot expansion
st.markdown("""
<script>
document.addEventListener('DOMContentLoaded', (event) => {
    document.querySelectorAll('.plot-card').forEach(card => {
        card.addEventListener('click', () => {
            card.classList.toggle('expanded');
        });
    });
});
</script>
""", unsafe_allow_html=True)

Writing app.py


In [ ]:
%%writefile time_series_derivatives_of_microbiome_dynamics_stlit_app.py
# -*- coding: utf-8 -*-
"""Time Series Derivatives of Microbiome Dynamics StLit App.ipynb

Automatically generated by Colab.

Original file is located at
    https://colab.research.google.com/drive/1Qe06MD86ADzqSefDHMbl8CiP8sxDP1Qd

16S sequencing

# MicrobiomeAnalysisPipeline Program Flow

## 1. Initialization and Data Preprocessing

The pipeline begins with the initialization of the `MicrobiomeAnalysisPipeline` class:

1. Load OTU (Operational Taxonomic Unit) data and metadata from CSV files.
2. Preprocess the data:
   - Align OTU data and metadata using common sample IDs.
   - Normalize OTU data.
   - Merge normalized OTU data with metadata.

## 2. Analysis Methods

The pipeline includes several analysis methods, each focusing on different aspects of the microbiome data:

### 2.1 Diel Patterns Analysis
- Visualize the abundance of top 5 OTUs over the course of a day.

### 2.2 Taxa Abundance Heatmap
- Generate a heatmap showing the abundance of the top 30 taxa across all samples.

### 2.3 Alpha Diversity Analysis
- Calculate and visualize Shannon diversity across samples.

### 2.4 Beta Diversity Analysis
- Perform Principal Coordinate Analysis (PCoA) on Bray-Curtis dissimilarity.
- Visualize samples in PCoA space, colored by time.

### 2.5 Taxonomic Composition
- Visualize the relative abundance of top taxa over time.

### 2.6 Time Derivative Analysis
- Calculate and visualize the rate of change in OTU abundances over time.

### 2.7 Smoothed Volatility Analysis
- Apply Savitzky-Golay filter to smooth OTU abundance data.
- Visualize the smoothed volatility in a heatmap.

### 2.8 Time Series Decomposition
- Decompose the time series of the most abundant OTU into trend, seasonal, and residual components.

### 2.9 Machine Learning Prediction
- Use Random Forest to predict time based on OTU abundances.
- Analyze feature importance and SHAP values.

## 3. Running the Pipeline

The `run_all_analyses()` method executes all the above analyses in sequence:

1. Diel patterns analysis
2. Taxa abundance heatmap
3. Alpha diversity analysis
4. Beta diversity analysis
5. Taxonomic composition
6. Time derivative analysis
7. Smoothed volatility analysis
8. Time series decomposition
9. Machine learning prediction

Installations and Imports
"""

from fastapi import FastAPI, File, UploadFile, Form, HTTPException, BackgroundTasks

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import zscore
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import plotly.express as px
import networkx as nx
from statsmodels.tsa.seasonal import STL
from scipy.signal import savgol_filter
# import rpy2.robjects as robjects
# from rpy2.robjects import pandas2ri
# from rpy2.robjects.packages import importr
import uuid
import os
# # Enable automatic conversion between R and Pandas dataframes
# pandas2ri.activate()

# # Import R's base package
# base = importr('base')

# """# Data Inspection

# """

import pandas as pd
import numpy as np

# Comprehensive 16S Analysis Pipeline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import zscore
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.inspection import permutation_importance
import plotly.express as px
import networkx as nx
from statsmodels.tsa.seasonal import STL
from scipy.signal import savgol_filter
from skbio.diversity import alpha_diversity, beta_diversity
from skbio.stats.ordination import pcoa
from scipy.spatial.distance import pdist, squareform
import shap

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import zscore
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.inspection import permutation_importance
import plotly.express as px
from statsmodels.tsa.seasonal import STL
from scipy.signal import savgol_filter
from skbio.diversity import alpha_diversity, beta_diversity
from skbio.stats.ordination import pcoa
from scipy.spatial.distance import pdist, squareform
import shap

class MicrobiomeAnalysisPipeline:
    def __init__(self, otu_file, metadata_file):
        self.otu_data = pd.read_csv(otu_file, index_col=0)
        self.metadata = pd.read_csv(metadata_file)
        self.preprocess_data()
        print("Data loaded and preprocessed successfully.")

    def preprocess_data(self):
        print("Preprocessing data...")
        self.metadata.set_index('sample_id', inplace=True)
        common_samples = self.otu_data.columns.intersection(self.metadata.index)
        self.otu_data = self.otu_data[common_samples]
        self.metadata = self.metadata.loc[common_samples]
        self.normalize_data()
        self.merged_data = pd.concat([self.normalized_data.T, self.metadata], axis=1)
        print("Metadata columns:", self.metadata.columns)
        print(f"Number of samples: {len(common_samples)}")
        print(f"Number of OTUs: {len(self.otu_data.index)}")

    def normalize_data(self):
        self.normalized_data = self.otu_data.div(self.otu_data.sum(axis=0), axis=1)

    def diel_patterns_analysis(self, filename):
        print("Performing diel patterns analysis...")
        time_of_day = (self.metadata['ind_time'] % 1) * 24
        top_otus = self.normalized_data.mean(axis=1).nlargest(5).index
        plt.figure(figsize=(12, 8))
        for otu in top_otus:
            plt.scatter(time_of_day, self.normalized_data.loc[otu], alpha=0.5, label=otu)
        plt.xlabel('Time of Day (hours)')
        plt.ylabel('Relative Abundance')
        plt.title('Diel Patterns of Top 5 OTUs')
        plt.legend()
        plt.savefig(filename)

    def taxa_abundance_heatmap(self, filename):
        print("Generating taxa abundance heatmap...")
        top_taxa = self.normalized_data.mean(axis=1).nlargest(30).index
        plt.figure(figsize=(15, 10))
        sns.heatmap(self.normalized_data.loc[top_taxa], cmap='viridis', xticklabels=False)
        plt.title('Abundance Heatmap of Top 30 Taxa')
        plt.xlabel('Samples')
        plt.ylabel('Taxa')
        plt.savefig(filename)

    def alpha_diversity_analysis(self, filename):
        print("Performing alpha diversity analysis...")
        shannon_diversity = alpha_diversity('shannon', self.otu_data.T)
        plt.figure(figsize=(12, 6))
        shannon_diversity.sort_values().plot(kind='bar')
        plt.title('Shannon Diversity Across Samples')
        plt.xlabel('Samples')
        plt.ylabel('Shannon Diversity')
        plt.xticks([])
        plt.tight_layout()
        plt.savefig()

    def beta_diversity_analysis(self, filename):
        print("Performing beta diversity analysis...")
        bc_dm = beta_diversity('braycurtis', self.otu_data.T)
        pc = pcoa(bc_dm)
        time_values = self.metadata['ind_time'].values

        # Print original shapes
        print(f"Original shape of PC1: {pc.samples['PC1'].shape}")
        print(f"Original shape of PC2: {pc.samples['PC2'].shape}")
        print(f"Original shape of time_values: {time_values.shape}")

        # Ensure time_values is 1D and matches the length of PC1 and PC2
        time_values = time_values.flatten()
        if len(time_values) < len(pc.samples['PC1']):
            # Pad time_values with the last value
            time_values = np.pad(time_values, (0, len(pc.samples['PC1']) - len(time_values)), 'edge')
        elif len(time_values) > len(pc.samples['PC1']):
            # Truncate time_values
            time_values = time_values[:len(pc.samples['PC1'])]

        # Print adjusted shapes
        print(f"Adjusted shape of time_values: {time_values.shape}")
        print(f"Shape of PC1: {pc.samples['PC1'].shape}")
        print(f"Shape of PC2: {pc.samples['PC2'].shape}")

        fig, ax = plt.subplots(figsize=(10, 8))
        scatter = ax.scatter(pc.samples['PC1'], pc.samples['PC2'], c=time_values, cmap='viridis')
        ax.set_xlabel(f'PC1 ({pc.proportion_explained[0]:.2%})')
        ax.set_ylabel(f'PC2 ({pc.proportion_explained[1]:.2%})')
        ax.set_title('PCoA of Bray-Curtis Dissimilarity')
        plt.colorbar(scatter, label='Time')
        plt.tight_layout()
        plt.savefig(filename)

        print(f"Number of samples in OTU data: {self.otu_data.shape[1]}")
        print(f"Number of samples in metadata: {len(self.metadata)}")

    def taxonomic_composition(self, filename, top_n=10):
        print(f"Visualizing top {top_n} taxa composition...")
        top_taxa = self.normalized_data.mean(axis=1).nlargest(top_n).index
        composition = self.normalized_data.loc[top_taxa].T
        fig = px.area(composition, title=f'Relative Abundance of Top {top_n} Taxa')
        fig.update_layout(legend_title='Taxa', xaxis_title='Samples', yaxis_title='Relative Abundance')
        fig.write_image(filename)

    def time_derivative_analysis(self, time_of_top_30, time_of_top_5):
        print("Performing time derivative analysis...")
        sorted_data = self.merged_data.sort_values(by='ind_time')
        time_diff = sorted_data['ind_time'].diff()
        derivatives = sorted_data.drop(self.metadata.columns, axis=1).diff().div(time_diff, axis=0)
        derivatives = derivatives.iloc[1:]
        top_otus = derivatives.abs().mean().nlargest(30).index
        plt.figure(figsize=(15, 10))
        sns.heatmap(derivatives[top_otus].T, cmap='RdBu_r', center=0, xticklabels=False, yticklabels=True)
        plt.title('Time Derivatives of Top 30 OTU Abundances')
        plt.xlabel('Samples (sorted by Time)')
        plt.ylabel('OTUs')
        plt.tight_layout()
        plt.savefig(time_of_top_30)
        plt.figure(figsize=(15, 10))
        for otu in top_otus[:5]:
            plt.plot(sorted_data['ind_time'][1:], derivatives[otu], label=otu)
        plt.title('Time Derivatives of Top 5 OTUs')
        plt.xlabel('Time')
        plt.ylabel('Derivative of Abundance')
        plt.legend()
        plt.tight_layout()
        plt.savefig(time_of_top_5)

    def smoothed_volatility_analysis(self, filename, window_length=5, polyorder=2):
        print("Performing smoothed volatility analysis...")

        # Ensure all columns have the same length
        max_length = max(self.normalized_data.apply(len))
        padded_data = self.normalized_data.apply(
            lambda col: np.pad(col, (0, max_length - len(col)),
                              'constant', constant_values=np.nan)
        )

        def safe_savgol_filter(x):
            # Remove NaN values
            valid_mask = ~np.isnan(x)
            valid_x = x[valid_mask]

            if len(valid_x) < window_length:
                return x  # Return original array if not enough valid data

            # Apply filter on valid data
            filtered = savgol_filter(valid_x, min(window_length, len(valid_x)), polyorder)

            # Put filtered data back in original array
            result = np.full_like(x, np.nan)
            result[valid_mask] = filtered
            return result

        # Apply safe Savitzky-Golay filter
        volatility = padded_data.apply(safe_savgol_filter)

        # Create a mask for NaN values
        mask = np.isnan(padded_data)

        plt.figure(figsize=(12, 8))
        sns.heatmap(volatility, cmap='viridis', mask=mask)
        plt.title('Smoothed Volatility (Savitzky-Golay filter)')
        plt.xlabel('Samples')
        plt.ylabel('OTUs')
        plt.savefig()

        print(f"Shape of volatility matrix: {volatility.shape}")
        print(f"Number of NaN values: {np.sum(mask)}")

    def time_series_decomposition(self, filename):
        print("Performing time series decomposition...")
        top_otu = self.normalized_data.mean(axis=1).idxmax()
        time_series = self.normalized_data.loc[top_otu]
        time_series.index = pd.to_datetime(self.metadata['ind_time'], unit='D')
        time_series = time_series.sort_index()
        time_series = time_series.resample('D').mean().interpolate()
        if len(time_series) < 14:
            print("Warning: Not enough data points for time series decomposition. Skipping this analysis.")
            return
        stl = STL(time_series, period=7)
        result = stl.fit()
        fig, (ax1, ax2, ax3, ax4) = plt.subplots(4, 1, figsize=(12, 16))
        ax1.plot(result.observed)
        ax1.set_title('Observed')
        ax2.plot(result.trend)
        ax2.set_title('Trend')
        ax3.plot(result.seasonal)
        ax3.set_title('Seasonal')
        ax4.plot(result.resid)
        ax4.set_title('Residual')
        plt.tight_layout()
        plt.savefig()

    def machine_learning_prediction(self):
        print("Performing machine learning prediction...")
        X = self.normalized_data.T
        y = self.metadata['ind_time']
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
        scaler = StandardScaler()
        X_train_scaled = scaler.fit_transform(X_train)
        X_test_scaled = scaler.transform(X_test)
        rf = RandomForestRegressor(n_estimators=100, random_state=42)
        rf.fit(X_train_scaled, y_train)
        y_pred = rf.predict(X_test_scaled)
        mse = mean_squared_error(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)
        print(f'Mean Squared Error: {mse}')
        print(f'R-squared Score: {r2}')
        # self.feature_importance_analysis(rf, X)
        self.shap_analysis(rf, X_test_scaled, X.columns)

    # def feature_importance_analysis(self, model, X):
    #     print("Performing feature importance analysis...")
    #     feature_importance = pd.Series(model.feature_importances_, index=X.columns).nlargest(10)
    #     plt.figure(figsize=(10, 6))
    #     feature_importance.plot(kind='bar')
    #     plt.title('Top 10 OTUs by Importance in Predicting Time')
    #     plt.xlabel('OTU')
    #     plt.ylabel('Importance')
    #     plt.tight_layout()
    #     plt.show()
    #     perm_importance = permutation_importance(model, X, self.metadata['ind_time'])
    #     perm_importance_sorted = pd.Series(perm_importance.importances_mean, index=X.columns).nlargest(10)
    #     plt.figure(figsize=(10, 6))
    #     perm_importance_sorted.plot(kind='bar')
    #     plt.title('Top 10 OTUs by Permutation Importance')
    #     plt.xlabel('OTU')
    #     plt.ylabel('Permutation Importance')
    #     plt.tight_layout()
    #     plt.show()

    def shap_analysis(self, model, X, feature_names, filename):
        print("Performing SHAP analysis...")
        explainer = shap.TreeExplainer(model)
        shap_values = explainer.shap_values(X)
        plt.figure(figsize=(10, 8))
        shap.summary_plot(shap_values, X, feature_names=feature_names, max_display=10)
        plt.tight_layout()
        plt.savefig(filename)

    def run_and_get_analyses(self):
        print("Running all analyses...")
        files = [f'diel_patterns_{uuid.uuid1()}.png', f'taxa_abundance_{uuid.uuid1()}.png', f'alpha_versity_{uuid.uuid1()}.png', f'beta_diversity_{uuid.uuid1()}.png', f'taxonomic_composition_{uuid.uuid1()}.png'
                 f'top_of_30_{uuid.uuid1()}.png', f'top_of_5_{uuid.uuid1()}.png', f'smoothed_volatility_{uuid.uuid1()}.png', f'timeseries_decomposition_{uuid.uuid1()}.png']
        for i in range(len(files)):
            files[i] = 'generated' + files[i]
        os.makedirs('generated', exist_ok=True)
        self.diel_patterns_analysis()
        self.taxa_abundance_heatmap()
        self.alpha_diversity_analysis()
        self.beta_diversity_analysis()
        self.taxonomic_composition()
        self.time_derivative_analysis()
        self.smoothed_volatility_analysis()
        self.time_series_decomposition()
        self.machine_learning_prediction()
        print("All analyses completed.")
        return files

Writing time_series_derivatives_of_microbiome_dynamics_stlit_app.py


In [ ]:
%%writefile lottie.json
{"v":"5.6.10","fr":30,"ip":30,"op":210,"w":800,"h":600,"nm":"Ã¥ÂÂÃ¦ÂÂ 1","ddd":0,"assets":[{"id":"comp_0","layers":[{"ddd":0,"ind":1,"ty":4,"nm":"Ã¥Â½Â¢Ã§ÂÂ¶Ã¥ÂÂ¾Ã¥Â±Â 18","sr":1,"ks":{"o":{"a":0,"k":100,"ix":11},"r":{"a":0,"k":340,"ix":10},"p":{"a":0,"k":[400,300,0],"ix":2},"a":{"a":0,"k":[1.258,2.078,0],"ix":1},"s":{"a":0,"k":[100,100,100],"ix":6}},"ao":0,"shapes":[{"ty":"gr","it":[{"d":1,"ty":"el","s":{"a":0,"k":[200,200],"ix":2},"p":{"a":0,"k":[0,0],"ix":3},"nm":"Ã¦Â¤Â­Ã¥ÂÂÃ¨Â·Â¯Ã¥Â¾Â 1","mn":"ADBE Vector Shape - Ellipse","hd":false},{"ty":"tm","s":{"a":0,"k":0,"ix":1},"e":{"a":0,"k":5,"ix":2},"o":{"a":1,"k":[{"i":{"x":[0.667],"y":[1]},"o":{"x":[0.333],"y":[0]},"t":0,"s":[0]},{"t":30,"s":[120]}],"ix":3},"m":1,"ix":2,"nm":"Ã¤Â¿Â®Ã¥ÂÂªÃ¨Â·Â¯Ã¥Â¾Â 1","mn":"ADBE Vector Filter - Trim","hd":false},{"ty":"st","c":{"a":0,"k":[0.09019607843137255,0.3843137254901961,0.8745098039215686,1],"ix":3},"o":{"a":0,"k":100,"ix":4},"w":{"a":0,"k":40,"ix":5},"lc":2,"lj":1,"ml":4,"bm":0,"nm":"Ã¦ÂÂÃ¨Â¾Â¹ 1","mn":"ADBE Vector Graphic - Stroke","hd":false},{"ty":"tr","p":{"a":0,"k":[1.258,2.078],"ix":2},"a":{"a":0,"k":[0,0],"ix":1},"s":{"a":0,"k":[100,100],"ix":3},"r":{"a":0,"k":0,"ix":6},"o":{"a":0,"k":100,"ix":7},"sk":{"a":0,"k":0,"ix":4},"sa":{"a":0,"k":0,"ix":5},"nm":"Ã¥ÂÂÃ¦ÂÂ¢"}],"nm":"Ã¦Â¤Â­Ã¥ÂÂ 1","np":4,"cix":2,"bm":0,"ix":1,"mn":"ADBE Vector Group","hd":false}],"ip":0,"op":121,"st":0,"bm":0},{"ddd":0,"ind":2,"ty":4,"nm":"Ã¥Â½Â¢Ã§ÂÂ¶Ã¥ÂÂ¾Ã¥Â±Â 17","sr":1,"ks":{"o":{"a":0,"k":100,"ix":11},"r":{"a":0,"k":320,"ix":10},"p":{"a":0,"k":[400,300,0],"ix":2},"a":{"a":0,"k":[1.258,2.078,0],"ix":1},"s":{"a":0,"k":[100,100,100],"ix":6}},"ao":0,"shapes":[{"ty":"gr","it":[{"d":1,"ty":"el","s":{"a":0,"k":[200,200],"ix":2},"p":{"a":0,"k":[0,0],"ix":3},"nm":"Ã¦Â¤Â­Ã¥ÂÂÃ¨Â·Â¯Ã¥Â¾Â 1","mn":"ADBE Vector Shape - Ellipse","hd":false},{"ty":"tm","s":{"a":0,"k":0,"ix":1},"e":{"a":0,"k":5,"ix":2},"o":{"a":1,"k":[{"i":{"x":[0.667],"y":[1]},"o":{"x":[0.333],"y":[0]},"t":10,"s":[0]},{"t":40,"s":[120]}],"ix":3},"m":1,"ix":2,"nm":"Ã¤Â¿Â®Ã¥ÂÂªÃ¨Â·Â¯Ã¥Â¾Â 1","mn":"ADBE Vector Filter - Trim","hd":false},{"ty":"st","c":{"a":0,"k":[0.09019607843137255,0.3843137254901961,0.8745098039215686,1],"ix":3},"o":{"a":0,"k":100,"ix":4},"w":{"a":0,"k":40,"ix":5},"lc":2,"lj":1,"ml":4,"bm":0,"nm":"Ã¦ÂÂÃ¨Â¾Â¹ 1","mn":"ADBE Vector Graphic - Stroke","hd":false},{"ty":"tr","p":{"a":0,"k":[1.258,2.078],"ix":2},"a":{"a":0,"k":[0,0],"ix":1},"s":{"a":0,"k":[100,100],"ix":3},"r":{"a":0,"k":0,"ix":6},"o":{"a":0,"k":100,"ix":7},"sk":{"a":0,"k":0,"ix":4},"sa":{"a":0,"k":0,"ix":5},"nm":"Ã¥ÂÂÃ¦ÂÂ¢"}],"nm":"Ã¦Â¤Â­Ã¥ÂÂ 1","np":4,"cix":2,"bm":0,"ix":1,"mn":"ADBE Vector Group","hd":false}],"ip":0,"op":121,"st":10,"bm":0},{"ddd":0,"ind":3,"ty":4,"nm":"Ã¥Â½Â¢Ã§ÂÂ¶Ã¥ÂÂ¾Ã¥Â±Â 16","sr":1,"ks":{"o":{"a":0,"k":100,"ix":11},"r":{"a":0,"k":300,"ix":10},"p":{"a":0,"k":[400,300,0],"ix":2},"a":{"a":0,"k":[1.258,2.078,0],"ix":1},"s":{"a":0,"k":[100,100,100],"ix":6}},"ao":0,"shapes":[{"ty":"gr","it":[{"d":1,"ty":"el","s":{"a":0,"k":[200,200],"ix":2},"p":{"a":0,"k":[0,0],"ix":3},"nm":"Ã¦Â¤Â­Ã¥ÂÂÃ¨Â·Â¯Ã¥Â¾Â 1","mn":"ADBE Vector Shape - Ellipse","hd":false},{"ty":"tm","s":{"a":0,"k":0,"ix":1},"e":{"a":0,"k":5,"ix":2},"o":{"a":1,"k":[{"i":{"x":[0.667],"y":[1]},"o":{"x":[0.333],"y":[0]},"t":20,"s":[0]},{"t":50,"s":[120]}],"ix":3},"m":1,"ix":2,"nm":"Ã¤Â¿Â®Ã¥ÂÂªÃ¨Â·Â¯Ã¥Â¾Â 1","mn":"ADBE Vector Filter - Trim","hd":false},{"ty":"st","c":{"a":0,"k":[0.09019607843137255,0.3843137254901961,0.8745098039215686,1],"ix":3},"o":{"a":0,"k":100,"ix":4},"w":{"a":0,"k":40,"ix":5},"lc":2,"lj":1,"ml":4,"bm":0,"nm":"Ã¦ÂÂÃ¨Â¾Â¹ 1","mn":"ADBE Vector Graphic - Stroke","hd":false},{"ty":"tr","p":{"a":0,"k":[1.258,2.078],"ix":2},"a":{"a":0,"k":[0,0],"ix":1},"s":{"a":0,"k":[100,100],"ix":3},"r":{"a":0,"k":0,"ix":6},"o":{"a":0,"k":100,"ix":7},"sk":{"a":0,"k":0,"ix":4},"sa":{"a":0,"k":0,"ix":5},"nm":"Ã¥ÂÂÃ¦ÂÂ¢"}],"nm":"Ã¦Â¤Â­Ã¥ÂÂ 1","np":4,"cix":2,"bm":0,"ix":1,"mn":"ADBE Vector Group","hd":false}],"ip":0,"op":121,"st":20,"bm":0},{"ddd":0,"ind":4,"ty":4,"nm":"Ã¥Â½Â¢Ã§ÂÂ¶Ã¥ÂÂ¾Ã¥Â±Â 15","sr":1,"ks":{"o":{"a":0,"k":100,"ix":11},"r":{"a":0,"k":280,"ix":10},"p":{"a":0,"k":[400,300,0],"ix":2},"a":{"a":0,"k":[1.258,2.078,0],"ix":1},"s":{"a":0,"k":[100,100,100],"ix":6}},"ao":0,"shapes":[{"ty":"gr","it":[{"d":1,"ty":"el","s":{"a":0,"k":[200,200],"ix":2},"p":{"a":0,"k":[0,0],"ix":3},"nm":"Ã¦Â¤Â­Ã¥ÂÂÃ¨Â·Â¯Ã¥Â¾Â 1","mn":"ADBE Vector Shape - Ellipse","hd":false},{"ty":"tm","s":{"a":0,"k":0,"ix":1},"e":{"a":0,"k":5,"ix":2},"o":{"a":1,"k":[{"i":{"x":[0.667],"y":[1]},"o":{"x":[0.333],"y":[0]},"t":30,"s":[0]},{"t":60,"s":[120]}],"ix":3},"m":1,"ix":2,"nm":"Ã¤Â¿Â®Ã¥ÂÂªÃ¨Â·Â¯Ã¥Â¾Â 1","mn":"ADBE Vector Filter - Trim","hd":false},{"ty":"st","c":{"a":0,"k":[0.09019607843137255,0.3843137254901961,0.8745098039215686,1],"ix":3},"o":{"a":0,"k":100,"ix":4},"w":{"a":0,"k":40,"ix":5},"lc":2,"lj":1,"ml":4,"bm":0,"nm":"Ã¦ÂÂÃ¨Â¾Â¹ 1","mn":"ADBE Vector Graphic - Stroke","hd":false},{"ty":"tr","p":{"a":0,"k":[1.258,2.078],"ix":2},"a":{"a":0,"k":[0,0],"ix":1},"s":{"a":0,"k":[100,100],"ix":3},"r":{"a":0,"k":0,"ix":6},"o":{"a":0,"k":100,"ix":7},"sk":{"a":0,"k":0,"ix":4},"sa":{"a":0,"k":0,"ix":5},"nm":"Ã¥ÂÂÃ¦ÂÂ¢"}],"nm":"Ã¦Â¤Â­Ã¥ÂÂ 1","np":4,"cix":2,"bm":0,"ix":1,"mn":"ADBE Vector Group","hd":false}],"ip":0,"op":121,"st":30,"bm":0},{"ddd":0,"ind":5,"ty":4,"nm":"Ã¥Â½Â¢Ã§ÂÂ¶Ã¥ÂÂ¾Ã¥Â±Â 14","sr":1,"ks":{"o":{"a":0,"k":100,"ix":11},"r":{"a":0,"k":260,"ix":10},"p":{"a":0,"k":[400,300,0],"ix":2},"a":{"a":0,"k":[1.258,2.078,0],"ix":1},"s":{"a":0,"k":[100,100,100],"ix":6}},"ao":0,"shapes":[{"ty":"gr","it":[{"d":1,"ty":"el","s":{"a":0,"k":[200,200],"ix":2},"p":{"a":0,"k":[0,0],"ix":3},"nm":"Ã¦Â¤Â­Ã¥ÂÂÃ¨Â·Â¯Ã¥Â¾Â 1","mn":"ADBE Vector Shape - Ellipse","hd":false},{"ty":"tm","s":{"a":0,"k":0,"ix":1},"e":{"a":0,"k":5,"ix":2},"o":{"a":1,"k":[{"i":{"x":[0.667],"y":[1]},"o":{"x":[0.333],"y":[0]},"t":40,"s":[0]},{"t":70,"s":[120]}],"ix":3},"m":1,"ix":2,"nm":"Ã¤Â¿Â®Ã¥ÂÂªÃ¨Â·Â¯Ã¥Â¾Â 1","mn":"ADBE Vector Filter - Trim","hd":false},{"ty":"st","c":{"a":0,"k":[0.09019607843137255,0.3843137254901961,0.8745098039215686,1],"ix":3},"o":{"a":0,"k":100,"ix":4},"w":{"a":0,"k":40,"ix":5},"lc":2,"lj":1,"ml":4,"bm":0,"nm":"Ã¦ÂÂÃ¨Â¾Â¹ 1","mn":"ADBE Vector Graphic - Stroke","hd":false},{"ty":"tr","p":{"a":0,"k":[1.258,2.078],"ix":2},"a":{"a":0,"k":[0,0],"ix":1},"s":{"a":0,"k":[100,100],"ix":3},"r":{"a":0,"k":0,"ix":6},"o":{"a":0,"k":100,"ix":7},"sk":{"a":0,"k":0,"ix":4},"sa":{"a":0,"k":0,"ix":5},"nm":"Ã¥ÂÂÃ¦ÂÂ¢"}],"nm":"Ã¦Â¤Â­Ã¥ÂÂ 1","np":4,"cix":2,"bm":0,"ix":1,"mn":"ADBE Vector Group","hd":false}],"ip":0,"op":121,"st":40,"bm":0},{"ddd":0,"ind":6,"ty":4,"nm":"Ã¥Â½Â¢Ã§ÂÂ¶Ã¥ÂÂ¾Ã¥Â±Â 13","sr":1,"ks":{"o":{"a":0,"k":100,"ix":11},"r":{"a":0,"k":240,"ix":10},"p":{"a":0,"k":[400,300,0],"ix":2},"a":{"a":0,"k":[1.258,2.078,0],"ix":1},"s":{"a":0,"k":[100,100,100],"ix":6}},"ao":0,"shapes":[{"ty":"gr","it":[{"d":1,"ty":"el","s":{"a":0,"k":[200,200],"ix":2},"p":{"a":0,"k":[0,0],"ix":3},"nm":"Ã¦Â¤Â­Ã¥ÂÂÃ¨Â·Â¯Ã¥Â¾Â 1","mn":"ADBE Vector Shape - Ellipse","hd":false},{"ty":"tm","s":{"a":0,"k":0,"ix":1},"e":{"a":0,"k":5,"ix":2},"o":{"a":1,"k":[{"i":{"x":[0.667],"y":[1]},"o":{"x":[0.333],"y":[0]},"t":50,"s":[0]},{"t":80,"s":[120]}],"ix":3},"m":1,"ix":2,"nm":"Ã¤Â¿Â®Ã¥ÂÂªÃ¨Â·Â¯Ã¥Â¾Â 1","mn":"ADBE Vector Filter - Trim","hd":false},{"ty":"st","c":{"a":0,"k":[0.09019607843137255,0.3843137254901961,0.8745098039215686,1],"ix":3},"o":{"a":0,"k":100,"ix":4},"w":{"a":0,"k":40,"ix":5},"lc":2,"lj":1,"ml":4,"bm":0,"nm":"Ã¦ÂÂÃ¨Â¾Â¹ 1","mn":"ADBE Vector Graphic - Stroke","hd":false},{"ty":"tr","p":{"a":0,"k":[1.258,2.078],"ix":2},"a":{"a":0,"k":[0,0],"ix":1},"s":{"a":0,"k":[100,100],"ix":3},"r":{"a":0,"k":0,"ix":6},"o":{"a":0,"k":100,"ix":7},"sk":{"a":0,"k":0,"ix":4},"sa":{"a":0,"k":0,"ix":5},"nm":"Ã¥ÂÂÃ¦ÂÂ¢"}],"nm":"Ã¦Â¤Â­Ã¥ÂÂ 1","np":4,"cix":2,"bm":0,"ix":1,"mn":"ADBE Vector Group","hd":false}],"ip":0,"op":121,"st":50,"bm":0},{"ddd":0,"ind":7,"ty":4,"nm":"Ã¥Â½Â¢Ã§ÂÂ¶Ã¥ÂÂ¾Ã¥Â±Â 12","sr":1,"ks":{"o":{"a":0,"k":100,"ix":11},"r":{"a":0,"k":220,"ix":10},"p":{"a":0,"k":[400,300,0],"ix":2},"a":{"a":0,"k":[1.258,2.078,0],"ix":1},"s":{"a":0,"k":[100,100,100],"ix":6}},"ao":0,"shapes":[{"ty":"gr","it":[{"d":1,"ty":"el","s":{"a":0,"k":[200,200],"ix":2},"p":{"a":0,"k":[0,0],"ix":3},"nm":"Ã¦Â¤Â­Ã¥ÂÂÃ¨Â·Â¯Ã¥Â¾Â 1","mn":"ADBE Vector Shape - Ellipse","hd":false},{"ty":"tm","s":{"a":0,"k":0,"ix":1},"e":{"a":0,"k":5,"ix":2},"o":{"a":1,"k":[{"i":{"x":[0.667],"y":[1]},"o":{"x":[0.333],"y":[0]},"t":60,"s":[0]},{"t":90,"s":[120]}],"ix":3},"m":1,"ix":2,"nm":"Ã¤Â¿Â®Ã¥ÂÂªÃ¨Â·Â¯Ã¥Â¾Â 1","mn":"ADBE Vector Filter - Trim","hd":false},{"ty":"st","c":{"a":0,"k":[0.09019607843137255,0.3843137254901961,0.8745098039215686,1],"ix":3},"o":{"a":0,"k":100,"ix":4},"w":{"a":0,"k":40,"ix":5},"lc":2,"lj":1,"ml":4,"bm":0,"nm":"Ã¦ÂÂÃ¨Â¾Â¹ 1","mn":"ADBE Vector Graphic - Stroke","hd":false},{"ty":"tr","p":{"a":0,"k":[1.258,2.078],"ix":2},"a":{"a":0,"k":[0,0],"ix":1},"s":{"a":0,"k":[100,100],"ix":3},"r":{"a":0,"k":0,"ix":6},"o":{"a":0,"k":100,"ix":7},"sk":{"a":0,"k":0,"ix":4},"sa":{"a":0,"k":0,"ix":5},"nm":"Ã¥ÂÂÃ¦ÂÂ¢"}],"nm":"Ã¦Â¤Â­Ã¥ÂÂ 1","np":4,"cix":2,"bm":0,"ix":1,"mn":"ADBE Vector Group","hd":false}],"ip":0,"op":181,"st":60,"bm":0},{"ddd":0,"ind":8,"ty":4,"nm":"Ã¥Â½Â¢Ã§ÂÂ¶Ã¥ÂÂ¾Ã¥Â±Â 11","sr":1,"ks":{"o":{"a":0,"k":100,"ix":11},"r":{"a":0,"k":200,"ix":10},"p":{"a":0,"k":[400,300,0],"ix":2},"a":{"a":0,"k":[1.258,2.078,0],"ix":1},"s":{"a":0,"k":[100,100,100],"ix":6}},"ao":0,"shapes":[{"ty":"gr","it":[{"d":1,"ty":"el","s":{"a":0,"k":[200,200],"ix":2},"p":{"a":0,"k":[0,0],"ix":3},"nm":"Ã¦Â¤Â­Ã¥ÂÂÃ¨Â·Â¯Ã¥Â¾Â 1","mn":"ADBE Vector Shape - Ellipse","hd":false},{"ty":"tm","s":{"a":0,"k":0,"ix":1},"e":{"a":0,"k":5,"ix":2},"o":{"a":1,"k":[{"i":{"x":[0.667],"y":[1]},"o":{"x":[0.333],"y":[0]},"t":70,"s":[0]},{"t":100,"s":[120]}],"ix":3},"m":1,"ix":2,"nm":"Ã¤Â¿Â®Ã¥ÂÂªÃ¨Â·Â¯Ã¥Â¾Â 1","mn":"ADBE Vector Filter - Trim","hd":false},{"ty":"st","c":{"a":0,"k":[0.09019607843137255,0.3843137254901961,0.8745098039215686,1],"ix":3},"o":{"a":0,"k":100,"ix":4},"w":{"a":0,"k":40,"ix":5},"lc":2,"lj":1,"ml":4,"bm":0,"nm":"Ã¦ÂÂÃ¨Â¾Â¹ 1","mn":"ADBE Vector Graphic - Stroke","hd":false},{"ty":"tr","p":{"a":0,"k":[1.258,2.078],"ix":2},"a":{"a":0,"k":[0,0],"ix":1},"s":{"a":0,"k":[100,100],"ix":3},"r":{"a":0,"k":0,"ix":6},"o":{"a":0,"k":100,"ix":7},"sk":{"a":0,"k":0,"ix":4},"sa":{"a":0,"k":0,"ix":5},"nm":"Ã¥ÂÂÃ¦ÂÂ¢"}],"nm":"Ã¦Â¤Â­Ã¥ÂÂ 1","np":4,"cix":2,"bm":0,"ix":1,"mn":"ADBE Vector Group","hd":false}],"ip":0,"op":181,"st":70,"bm":0},{"ddd":0,"ind":9,"ty":4,"nm":"Ã¥Â½Â¢Ã§ÂÂ¶Ã¥ÂÂ¾Ã¥Â±Â 10","sr":1,"ks":{"o":{"a":0,"k":100,"ix":11},"r":{"a":0,"k":180,"ix":10},"p":{"a":0,"k":[400,300,0],"ix":2},"a":{"a":0,"k":[1.258,2.078,0],"ix":1},"s":{"a":0,"k":[100,100,100],"ix":6}},"ao":0,"shapes":[{"ty":"gr","it":[{"d":1,"ty":"el","s":{"a":0,"k":[200,200],"ix":2},"p":{"a":0,"k":[0,0],"ix":3},"nm":"Ã¦Â¤Â­Ã¥ÂÂÃ¨Â·Â¯Ã¥Â¾Â 1","mn":"ADBE Vector Shape - Ellipse","hd":false},{"ty":"tm","s":{"a":0,"k":0,"ix":1},"e":{"a":0,"k":5,"ix":2},"o":{"a":1,"k":[{"i":{"x":[0.667],"y":[1]},"o":{"x":[0.333],"y":[0]},"t":80,"s":[0]},{"t":110,"s":[120]}],"ix":3},"m":1,"ix":2,"nm":"Ã¤Â¿Â®Ã¥ÂÂªÃ¨Â·Â¯Ã¥Â¾Â 1","mn":"ADBE Vector Filter - Trim","hd":false},{"ty":"st","c":{"a":0,"k":[0.09019607843137255,0.3843137254901961,0.8745098039215686,1],"ix":3},"o":{"a":0,"k":100,"ix":4},"w":{"a":0,"k":40,"ix":5},"lc":2,"lj":1,"ml":4,"bm":0,"nm":"Ã¦ÂÂÃ¨Â¾Â¹ 1","mn":"ADBE Vector Graphic - Stroke","hd":false},{"ty":"tr","p":{"a":0,"k":[1.258,2.078],"ix":2},"a":{"a":0,"k":[0,0],"ix":1},"s":{"a":0,"k":[100,100],"ix":3},"r":{"a":0,"k":0,"ix":6},"o":{"a":0,"k":100,"ix":7},"sk":{"a":0,"k":0,"ix":4},"sa":{"a":0,"k":0,"ix":5},"nm":"Ã¥ÂÂÃ¦ÂÂ¢"}],"nm":"Ã¦Â¤Â­Ã¥ÂÂ 1","np":4,"cix":2,"bm":0,"ix":1,"mn":"ADBE Vector Group","hd":false}],"ip":0,"op":181,"st":80,"bm":0},{"ddd":0,"ind":10,"ty":4,"nm":"Ã¥Â½Â¢Ã§ÂÂ¶Ã¥ÂÂ¾Ã¥Â±Â 9","sr":1,"ks":{"o":{"a":0,"k":100,"ix":11},"r":{"a":0,"k":160,"ix":10},"p":{"a":0,"k":[400,300,0],"ix":2},"a":{"a":0,"k":[1.258,2.078,0],"ix":1},"s":{"a":0,"k":[100,100,100],"ix":6}},"ao":0,"shapes":[{"ty":"gr","it":[{"d":1,"ty":"el","s":{"a":0,"k":[200,200],"ix":2},"p":{"a":0,"k":[0,0],"ix":3},"nm":"Ã¦Â¤Â­Ã¥ÂÂÃ¨Â·Â¯Ã¥Â¾Â 1","mn":"ADBE Vector Shape - Ellipse","hd":false},{"ty":"tm","s":{"a":0,"k":0,"ix":1},"e":{"a":0,"k":5,"ix":2},"o":{"a":1,"k":[{"i":{"x":[0.667],"y":[1]},"o":{"x":[0.333],"y":[0]},"t":90,"s":[0]},{"t":120,"s":[120]}],"ix":3},"m":1,"ix":2,"nm":"Ã¤Â¿Â®Ã¥ÂÂªÃ¨Â·Â¯Ã¥Â¾Â 1","mn":"ADBE Vector Filter - Trim","hd":false},{"ty":"st","c":{"a":0,"k":[0.09019607843137255,0.3843137254901961,0.8745098039215686,1],"ix":3},"o":{"a":0,"k":100,"ix":4},"w":{"a":0,"k":40,"ix":5},"lc":2,"lj":1,"ml":4,"bm":0,"nm":"Ã¦ÂÂÃ¨Â¾Â¹ 1","mn":"ADBE Vector Graphic - Stroke","hd":false},{"ty":"tr","p":{"a":0,"k":[1.258,2.078],"ix":2},"a":{"a":0,"k":[0,0],"ix":1},"s":{"a":0,"k":[100,100],"ix":3},"r":{"a":0,"k":0,"ix":6},"o":{"a":0,"k":100,"ix":7},"sk":{"a":0,"k":0,"ix":4},"sa":{"a":0,"k":0,"ix":5},"nm":"Ã¥ÂÂÃ¦ÂÂ¢"}],"nm":"Ã¦Â¤Â­Ã¥ÂÂ 1","np":4,"cix":2,"bm":0,"ix":1,"mn":"ADBE Vector Group","hd":false}],"ip":0,"op":181,"st":90,"bm":0},{"ddd":0,"ind":11,"ty":4,"nm":"Ã¥Â½Â¢Ã§ÂÂ¶Ã¥ÂÂ¾Ã¥Â±Â 8","sr":1,"ks":{"o":{"a":0,"k":100,"ix":11},"r":{"a":0,"k":140,"ix":10},"p":{"a":0,"k":[400,300,0],"ix":2},"a":{"a":0,"k":[1.258,2.078,0],"ix":1},"s":{"a":0,"k":[100,100,100],"ix":6}},"ao":0,"shapes":[{"ty":"gr","it":[{"d":1,"ty":"el","s":{"a":0,"k":[200,200],"ix":2},"p":{"a":0,"k":[0,0],"ix":3},"nm":"Ã¦Â¤Â­Ã¥ÂÂÃ¨Â·Â¯Ã¥Â¾Â 1","mn":"ADBE Vector Shape - Ellipse","hd":false},{"ty":"tm","s":{"a":0,"k":0,"ix":1},"e":{"a":0,"k":5,"ix":2},"o":{"a":1,"k":[{"i":{"x":[0.667],"y":[1]},"o":{"x":[0.333],"y":[0]},"t":100,"s":[0]},{"t":130,"s":[120]}],"ix":3},"m":1,"ix":2,"nm":"Ã¤Â¿Â®Ã¥ÂÂªÃ¨Â·Â¯Ã¥Â¾Â 1","mn":"ADBE Vector Filter - Trim","hd":false},{"ty":"st","c":{"a":0,"k":[0.09019607843137255,0.3843137254901961,0.8745098039215686,1],"ix":3},"o":{"a":0,"k":100,"ix":4},"w":{"a":0,"k":40,"ix":5},"lc":2,"lj":1,"ml":4,"bm":0,"nm":"Ã¦ÂÂÃ¨Â¾Â¹ 1","mn":"ADBE Vector Graphic - Stroke","hd":false},{"ty":"tr","p":{"a":0,"k":[1.258,2.078],"ix":2},"a":{"a":0,"k":[0,0],"ix":1},"s":{"a":0,"k":[100,100],"ix":3},"r":{"a":0,"k":0,"ix":6},"o":{"a":0,"k":100,"ix":7},"sk":{"a":0,"k":0,"ix":4},"sa":{"a":0,"k":0,"ix":5},"nm":"Ã¥ÂÂÃ¦ÂÂ¢"}],"nm":"Ã¦Â¤Â­Ã¥ÂÂ 1","np":4,"cix":2,"bm":0,"ix":1,"mn":"ADBE Vector Group","hd":false}],"ip":0,"op":181,"st":100,"bm":0},{"ddd":0,"ind":12,"ty":4,"nm":"Ã¥Â½Â¢Ã§ÂÂ¶Ã¥ÂÂ¾Ã¥Â±Â 7","sr":1,"ks":{"o":{"a":0,"k":100,"ix":11},"r":{"a":0,"k":120,"ix":10},"p":{"a":0,"k":[400,300,0],"ix":2},"a":{"a":0,"k":[1.258,2.078,0],"ix":1},"s":{"a":0,"k":[100,100,100],"ix":6}},"ao":0,"shapes":[{"ty":"gr","it":[{"d":1,"ty":"el","s":{"a":0,"k":[200,200],"ix":2},"p":{"a":0,"k":[0,0],"ix":3},"nm":"Ã¦Â¤Â­Ã¥ÂÂÃ¨Â·Â¯Ã¥Â¾Â 1","mn":"ADBE Vector Shape - Ellipse","hd":false},{"ty":"tm","s":{"a":0,"k":0,"ix":1},"e":{"a":0,"k":5,"ix":2},"o":{"a":1,"k":[{"i":{"x":[0.667],"y":[1]},"o":{"x":[0.333],"y":[0]},"t":110,"s":[0]},{"t":140,"s":[120]}],"ix":3},"m":1,"ix":2,"nm":"Ã¤Â¿Â®Ã¥ÂÂªÃ¨Â·Â¯Ã¥Â¾Â 1","mn":"ADBE Vector Filter - Trim","hd":false},{"ty":"st","c":{"a":0,"k":[0.09019607843137255,0.3843137254901961,0.8745098039215686,1],"ix":3},"o":{"a":0,"k":100,"ix":4},"w":{"a":0,"k":40,"ix":5},"lc":2,"lj":1,"ml":4,"bm":0,"nm":"Ã¦ÂÂÃ¨Â¾Â¹ 1","mn":"ADBE Vector Graphic - Stroke","hd":false},{"ty":"tr","p":{"a":0,"k":[1.258,2.078],"ix":2},"a":{"a":0,"k":[0,0],"ix":1},"s":{"a":0,"k":[100,100],"ix":3},"r":{"a":0,"k":0,"ix":6},"o":{"a":0,"k":100,"ix":7},"sk":{"a":0,"k":0,"ix":4},"sa":{"a":0,"k":0,"ix":5},"nm":"Ã¥ÂÂÃ¦ÂÂ¢"}],"nm":"Ã¦Â¤Â­Ã¥ÂÂ 1","np":4,"cix":2,"bm":0,"ix":1,"mn":"ADBE Vector Group","hd":false}],"ip":0,"op":181,"st":110,"bm":0},{"ddd":0,"ind":13,"ty":4,"nm":"Ã¥Â½Â¢Ã§ÂÂ¶Ã¥ÂÂ¾Ã¥Â±Â 6","sr":1,"ks":{"o":{"a":0,"k":100,"ix":11},"r":{"a":0,"k":100,"ix":10},"p":{"a":0,"k":[400,300,0],"ix":2},"a":{"a":0,"k":[1.258,2.078,0],"ix":1},"s":{"a":0,"k":[100,100,100],"ix":6}},"ao":0,"shapes":[{"ty":"gr","it":[{"d":1,"ty":"el","s":{"a":0,"k":[200,200],"ix":2},"p":{"a":0,"k":[0,0],"ix":3},"nm":"Ã¦Â¤Â­Ã¥ÂÂÃ¨Â·Â¯Ã¥Â¾Â 1","mn":"ADBE Vector Shape - Ellipse","hd":false},{"ty":"tm","s":{"a":0,"k":0,"ix":1},"e":{"a":0,"k":5,"ix":2},"o":{"a":1,"k":[{"i":{"x":[0.667],"y":[1]},"o":{"x":[0.333],"y":[0]},"t":120,"s":[0]},{"t":150,"s":[120]}],"ix":3},"m":1,"ix":2,"nm":"Ã¤Â¿Â®Ã¥ÂÂªÃ¨Â·Â¯Ã¥Â¾Â 1","mn":"ADBE Vector Filter - Trim","hd":false},{"ty":"st","c":{"a":0,"k":[0.09019607843137255,0.3843137254901961,0.8745098039215686,1],"ix":3},"o":{"a":0,"k":100,"ix":4},"w":{"a":0,"k":40,"ix":5},"lc":2,"lj":1,"ml":4,"bm":0,"nm":"Ã¦ÂÂÃ¨Â¾Â¹ 1","mn":"ADBE Vector Graphic - Stroke","hd":false},{"ty":"tr","p":{"a":0,"k":[1.258,2.078],"ix":2},"a":{"a":0,"k":[0,0],"ix":1},"s":{"a":0,"k":[100,100],"ix":3},"r":{"a":0,"k":0,"ix":6},"o":{"a":0,"k":100,"ix":7},"sk":{"a":0,"k":0,"ix":4},"sa":{"a":0,"k":0,"ix":5},"nm":"Ã¥ÂÂÃ¦ÂÂ¢"}],"nm":"Ã¦Â¤Â­Ã¥ÂÂ 1","np":4,"cix":2,"bm":0,"ix":1,"mn":"ADBE Vector Group","hd":false}],"ip":78,"op":420,"st":120,"bm":0},{"ddd":0,"ind":14,"ty":4,"nm":"Ã¥Â½Â¢Ã§ÂÂ¶Ã¥ÂÂ¾Ã¥Â±Â 5","sr":1,"ks":{"o":{"a":0,"k":100,"ix":11},"r":{"a":0,"k":80,"ix":10},"p":{"a":0,"k":[400,300,0],"ix":2},"a":{"a":0,"k":[1.258,2.078,0],"ix":1},"s":{"a":0,"k":[100,100,100],"ix":6}},"ao":0,"shapes":[{"ty":"gr","it":[{"d":1,"ty":"el","s":{"a":0,"k":[200,200],"ix":2},"p":{"a":0,"k":[0,0],"ix":3},"nm":"Ã¦Â¤Â­Ã¥ÂÂÃ¨Â·Â¯Ã¥Â¾Â 1","mn":"ADBE Vector Shape - Ellipse","hd":false},{"ty":"tm","s":{"a":0,"k":0,"ix":1},"e":{"a":0,"k":5,"ix":2},"o":{"a":1,"k":[{"i":{"x":[0.667],"y":[1]},"o":{"x":[0.333],"y":[0]},"t":130,"s":[0]},{"t":160,"s":[120]}],"ix":3},"m":1,"ix":2,"nm":"Ã¤Â¿Â®Ã¥ÂÂªÃ¨Â·Â¯Ã¥Â¾Â 1","mn":"ADBE Vector Filter - Trim","hd":false},{"ty":"st","c":{"a":0,"k":[0.09019607843137255,0.3843137254901961,0.8745098039215686,1],"ix":3},"o":{"a":0,"k":100,"ix":4},"w":{"a":0,"k":40,"ix":5},"lc":2,"lj":1,"ml":4,"bm":0,"nm":"Ã¦ÂÂÃ¨Â¾Â¹ 1","mn":"ADBE Vector Graphic - Stroke","hd":false},{"ty":"tr","p":{"a":0,"k":[1.258,2.078],"ix":2},"a":{"a":0,"k":[0,0],"ix":1},"s":{"a":0,"k":[100,100],"ix":3},"r":{"a":0,"k":0,"ix":6},"o":{"a":0,"k":100,"ix":7},"sk":{"a":0,"k":0,"ix":4},"sa":{"a":0,"k":0,"ix":5},"nm":"Ã¥ÂÂÃ¦ÂÂ¢"}],"nm":"Ã¦Â¤Â­Ã¥ÂÂ 1","np":4,"cix":2,"bm":0,"ix":1,"mn":"ADBE Vector Group","hd":false}],"ip":78,"op":430,"st":130,"bm":0},{"ddd":0,"ind":15,"ty":4,"nm":"Ã¥Â½Â¢Ã§ÂÂ¶Ã¥ÂÂ¾Ã¥Â±Â 4","sr":1,"ks":{"o":{"a":0,"k":100,"ix":11},"r":{"a":0,"k":60,"ix":10},"p":{"a":0,"k":[400,300,0],"ix":2},"a":{"a":0,"k":[1.258,2.078,0],"ix":1},"s":{"a":0,"k":[100,100,100],"ix":6}},"ao":0,"shapes":[{"ty":"gr","it":[{"d":1,"ty":"el","s":{"a":0,"k":[200,200],"ix":2},"p":{"a":0,"k":[0,0],"ix":3},"nm":"Ã¦Â¤Â­Ã¥ÂÂÃ¨Â·Â¯Ã¥Â¾Â 1","mn":"ADBE Vector Shape - Ellipse","hd":false},{"ty":"tm","s":{"a":0,"k":0,"ix":1},"e":{"a":0,"k":5,"ix":2},"o":{"a":1,"k":[{"i":{"x":[0.667],"y":[1]},"o":{"x":[0.333],"y":[0]},"t":140,"s":[0]},{"t":170,"s":[120]}],"ix":3},"m":1,"ix":2,"nm":"Ã¤Â¿Â®Ã¥ÂÂªÃ¨Â·Â¯Ã¥Â¾Â 1","mn":"ADBE Vector Filter - Trim","hd":false},{"ty":"st","c":{"a":0,"k":[0.09019607843137255,0.3843137254901961,0.8745098039215686,1],"ix":3},"o":{"a":0,"k":100,"ix":4},"w":{"a":0,"k":40,"ix":5},"lc":2,"lj":1,"ml":4,"bm":0,"nm":"Ã¦ÂÂÃ¨Â¾Â¹ 1","mn":"ADBE Vector Graphic - Stroke","hd":false},{"ty":"tr","p":{"a":0,"k":[1.258,2.078],"ix":2},"a":{"a":0,"k":[0,0],"ix":1},"s":{"a":0,"k":[100,100],"ix":3},"r":{"a":0,"k":0,"ix":6},"o":{"a":0,"k":100,"ix":7},"sk":{"a":0,"k":0,"ix":4},"sa":{"a":0,"k":0,"ix":5},"nm":"Ã¥ÂÂÃ¦ÂÂ¢"}],"nm":"Ã¦Â¤Â­Ã¥ÂÂ 1","np":4,"cix":2,"bm":0,"ix":1,"mn":"ADBE Vector Group","hd":false}],"ip":78,"op":440,"st":140,"bm":0},{"ddd":0,"ind":16,"ty":4,"nm":"Ã¥Â½Â¢Ã§ÂÂ¶Ã¥ÂÂ¾Ã¥Â±Â 3","sr":1,"ks":{"o":{"a":0,"k":100,"ix":11},"r":{"a":0,"k":40,"ix":10},"p":{"a":0,"k":[400,300,0],"ix":2},"a":{"a":0,"k":[1.258,2.078,0],"ix":1},"s":{"a":0,"k":[100,100,100],"ix":6}},"ao":0,"shapes":[{"ty":"gr","it":[{"d":1,"ty":"el","s":{"a":0,"k":[200,200],"ix":2},"p":{"a":0,"k":[0,0],"ix":3},"nm":"Ã¦Â¤Â­Ã¥ÂÂÃ¨Â·Â¯Ã¥Â¾Â 1","mn":"ADBE Vector Shape - Ellipse","hd":false},{"ty":"tm","s":{"a":0,"k":0,"ix":1},"e":{"a":0,"k":5,"ix":2},"o":{"a":1,"k":[{"i":{"x":[0.667],"y":[1]},"o":{"x":[0.333],"y":[0]},"t":150,"s":[0]},{"t":180,"s":[120]}],"ix":3},"m":1,"ix":2,"nm":"Ã¤Â¿Â®Ã¥ÂÂªÃ¨Â·Â¯Ã¥Â¾Â 1","mn":"ADBE Vector Filter - Trim","hd":false},{"ty":"st","c":{"a":0,"k":[0.09019607843137255,0.3843137254901961,0.8745098039215686,1],"ix":3},"o":{"a":0,"k":100,"ix":4},"w":{"a":0,"k":40,"ix":5},"lc":2,"lj":1,"ml":4,"bm":0,"nm":"Ã¦ÂÂÃ¨Â¾Â¹ 1","mn":"ADBE Vector Graphic - Stroke","hd":false},{"ty":"tr","p":{"a":0,"k":[1.258,2.078],"ix":2},"a":{"a":0,"k":[0,0],"ix":1},"s":{"a":0,"k":[100,100],"ix":3},"r":{"a":0,"k":0,"ix":6},"o":{"a":0,"k":100,"ix":7},"sk":{"a":0,"k":0,"ix":4},"sa":{"a":0,"k":0,"ix":5},"nm":"Ã¥ÂÂÃ¦ÂÂ¢"}],"nm":"Ã¦Â¤Â­Ã¥ÂÂ 1","np":4,"cix":2,"bm":0,"ix":1,"mn":"ADBE Vector Group","hd":false}],"ip":78,"op":450,"st":150,"bm":0},{"ddd":0,"ind":17,"ty":4,"nm":"Ã¥Â½Â¢Ã§ÂÂ¶Ã¥ÂÂ¾Ã¥Â±Â 2","sr":1,"ks":{"o":{"a":0,"k":100,"ix":11},"r":{"a":0,"k":20,"ix":10},"p":{"a":0,"k":[400,300,0],"ix":2},"a":{"a":0,"k":[1.258,2.078,0],"ix":1},"s":{"a":0,"k":[100,100,100],"ix":6}},"ao":0,"shapes":[{"ty":"gr","it":[{"d":1,"ty":"el","s":{"a":0,"k":[200,200],"ix":2},"p":{"a":0,"k":[0,0],"ix":3},"nm":"Ã¦Â¤Â­Ã¥ÂÂÃ¨Â·Â¯Ã¥Â¾Â 1","mn":"ADBE Vector Shape - Ellipse","hd":false},{"ty":"tm","s":{"a":0,"k":0,"ix":1},"e":{"a":0,"k":5,"ix":2},"o":{"a":1,"k":[{"i":{"x":[0.667],"y":[1]},"o":{"x":[0.333],"y":[0]},"t":160,"s":[0]},{"t":190,"s":[120]}],"ix":3},"m":1,"ix":2,"nm":"Ã¤Â¿Â®Ã¥ÂÂªÃ¨Â·Â¯Ã¥Â¾Â 1","mn":"ADBE Vector Filter - Trim","hd":false},{"ty":"st","c":{"a":0,"k":[0.09019607843137255,0.3843137254901961,0.8745098039215686,1],"ix":3},"o":{"a":0,"k":100,"ix":4},"w":{"a":0,"k":40,"ix":5},"lc":2,"lj":1,"ml":4,"bm":0,"nm":"Ã¦ÂÂÃ¨Â¾Â¹ 1","mn":"ADBE Vector Graphic - Stroke","hd":false},{"ty":"tr","p":{"a":0,"k":[1.258,2.078],"ix":2},"a":{"a":0,"k":[0,0],"ix":1},"s":{"a":0,"k":[100,100],"ix":3},"r":{"a":0,"k":0,"ix":6},"o":{"a":0,"k":100,"ix":7},"sk":{"a":0,"k":0,"ix":4},"sa":{"a":0,"k":0,"ix":5},"nm":"Ã¥ÂÂÃ¦ÂÂ¢"}],"nm":"Ã¦Â¤Â­Ã¥ÂÂ 1","np":4,"cix":2,"bm":0,"ix":1,"mn":"ADBE Vector Group","hd":false}],"ip":78,"op":460,"st":160,"bm":0},{"ddd":0,"ind":18,"ty":4,"nm":"Ã¥Â½Â¢Ã§ÂÂ¶Ã¥ÂÂ¾Ã¥Â±Â 1","sr":1,"ks":{"o":{"a":0,"k":100,"ix":11},"r":{"a":0,"k":0,"ix":10},"p":{"a":0,"k":[400,300,0],"ix":2},"a":{"a":0,"k":[1.258,2.078,0],"ix":1},"s":{"a":0,"k":[100,100,100],"ix":6}},"ao":0,"shapes":[{"ty":"gr","it":[{"d":1,"ty":"el","s":{"a":0,"k":[200,200],"ix":2},"p":{"a":0,"k":[0,0],"ix":3},"nm":"Ã¦Â¤Â­Ã¥ÂÂÃ¨Â·Â¯Ã¥Â¾Â 1","mn":"ADBE Vector Shape - Ellipse","hd":false},{"ty":"tm","s":{"a":0,"k":0,"ix":1},"e":{"a":0,"k":5,"ix":2},"o":{"a":1,"k":[{"i":{"x":[0.667],"y":[1]},"o":{"x":[0.333],"y":[0]},"t":170,"s":[0]},{"t":200,"s":[120]}],"ix":3},"m":1,"ix":2,"nm":"Ã¤Â¿Â®Ã¥ÂÂªÃ¨Â·Â¯Ã¥Â¾Â 1","mn":"ADBE Vector Filter - Trim","hd":false},{"ty":"st","c":{"a":0,"k":[0.09019607843137255,0.3843137254901961,0.8745098039215686,1],"ix":3},"o":{"a":0,"k":100,"ix":4},"w":{"a":0,"k":40,"ix":5},"lc":2,"lj":1,"ml":4,"bm":0,"nm":"Ã¦ÂÂÃ¨Â¾Â¹ 1","mn":"ADBE Vector Graphic - Stroke","hd":false},{"ty":"tr","p":{"a":0,"k":[1.258,2.078],"ix":2},"a":{"a":0,"k":[0,0],"ix":1},"s":{"a":0,"k":[100,100],"ix":3},"r":{"a":0,"k":0,"ix":6},"o":{"a":0,"k":100,"ix":7},"sk":{"a":0,"k":0,"ix":4},"sa":{"a":0,"k":0,"ix":5},"nm":"Ã¥ÂÂÃ¦ÂÂ¢"}],"nm":"Ã¦Â¤Â­Ã¥ÂÂ 1","np":4,"cix":2,"bm":0,"ix":1,"mn":"ADBE Vector Group","hd":false}],"ip":78,"op":470,"st":170,"bm":0},{"ddd":0,"ind":19,"ty":4,"nm":"Ã¥Â½Â¢Ã§ÂÂ¶Ã¥ÂÂ¾Ã¥Â±Â 24","sr":1,"ks":{"o":{"a":0,"k":100,"ix":11},"r":{"a":0,"k":340,"ix":10},"p":{"a":0,"k":[400,300,0],"ix":2},"a":{"a":0,"k":[1.258,2.078,0],"ix":1},"s":{"a":0,"k":[100,100,100],"ix":6}},"ao":0,"shapes":[{"ty":"gr","it":[{"d":1,"ty":"el","s":{"a":0,"k":[200,200],"ix":2},"p":{"a":0,"k":[0,0],"ix":3},"nm":"Ã¦Â¤Â­Ã¥ÂÂÃ¨Â·Â¯Ã¥Â¾Â 1","mn":"ADBE Vector Shape - Ellipse","hd":false},{"ty":"tm","s":{"a":0,"k":0,"ix":1},"e":{"a":0,"k":5,"ix":2},"o":{"a":1,"k":[{"i":{"x":[0.667],"y":[1]},"o":{"x":[0.333],"y":[0]},"t":180,"s":[0]},{"t":210,"s":[120]}],"ix":3},"m":1,"ix":2,"nm":"Ã¤Â¿Â®Ã¥ÂÂªÃ¨Â·Â¯Ã¥Â¾Â 1","mn":"ADBE Vector Filter - Trim","hd":false},{"ty":"st","c":{"a":0,"k":[0.09019607843137255,0.3843137254901961,0.8745098039215686,1],"ix":3},"o":{"a":0,"k":100,"ix":4},"w":{"a":0,"k":40,"ix":5},"lc":2,"lj":1,"ml":4,"bm":0,"nm":"Ã¦ÂÂÃ¨Â¾Â¹ 1","mn":"ADBE Vector Graphic - Stroke","hd":false},{"ty":"tr","p":{"a":0,"k":[1.258,2.078],"ix":2},"a":{"a":0,"k":[0,0],"ix":1},"s":{"a":0,"k":[100,100],"ix":3},"r":{"a":0,"k":0,"ix":6},"o":{"a":0,"k":100,"ix":7},"sk":{"a":0,"k":0,"ix":4},"sa":{"a":0,"k":0,"ix":5},"nm":"Ã¥ÂÂÃ¦ÂÂ¢"}],"nm":"Ã¦Â¤Â­Ã¥ÂÂ 1","np":4,"cix":2,"bm":0,"ix":1,"mn":"ADBE Vector Group","hd":false}],"ip":180,"op":301,"st":180,"bm":0},{"ddd":0,"ind":20,"ty":4,"nm":"Ã¥Â½Â¢Ã§ÂÂ¶Ã¥ÂÂ¾Ã¥Â±Â 23","sr":1,"ks":{"o":{"a":0,"k":100,"ix":11},"r":{"a":0,"k":320,"ix":10},"p":{"a":0,"k":[400,300,0],"ix":2},"a":{"a":0,"k":[1.258,2.078,0],"ix":1},"s":{"a":0,"k":[100,100,100],"ix":6}},"ao":0,"shapes":[{"ty":"gr","it":[{"d":1,"ty":"el","s":{"a":0,"k":[200,200],"ix":2},"p":{"a":0,"k":[0,0],"ix":3},"nm":"Ã¦Â¤Â­Ã¥ÂÂÃ¨Â·Â¯Ã¥Â¾Â 1","mn":"ADBE Vector Shape - Ellipse","hd":false},{"ty":"tm","s":{"a":0,"k":0,"ix":1},"e":{"a":0,"k":5,"ix":2},"o":{"a":1,"k":[{"i":{"x":[0.667],"y":[1]},"o":{"x":[0.333],"y":[0]},"t":190,"s":[0]},{"t":220,"s":[120]}],"ix":3},"m":1,"ix":2,"nm":"Ã¤Â¿Â®Ã¥ÂÂªÃ¨Â·Â¯Ã¥Â¾Â 1","mn":"ADBE Vector Filter - Trim","hd":false},{"ty":"st","c":{"a":0,"k":[0.09019607843137255,0.3843137254901961,0.8745098039215686,1],"ix":3},"o":{"a":0,"k":100,"ix":4},"w":{"a":0,"k":40,"ix":5},"lc":2,"lj":1,"ml":4,"bm":0,"nm":"Ã¦ÂÂÃ¨Â¾Â¹ 1","mn":"ADBE Vector Graphic - Stroke","hd":false},{"ty":"tr","p":{"a":0,"k":[1.258,2.078],"ix":2},"a":{"a":0,"k":[0,0],"ix":1},"s":{"a":0,"k":[100,100],"ix":3},"r":{"a":0,"k":0,"ix":6},"o":{"a":0,"k":100,"ix":7},"sk":{"a":0,"k":0,"ix":4},"sa":{"a":0,"k":0,"ix":5},"nm":"Ã¥ÂÂÃ¦ÂÂ¢"}],"nm":"Ã¦Â¤Â­Ã¥ÂÂ 1","np":4,"cix":2,"bm":0,"ix":1,"mn":"ADBE Vector Group","hd":false}],"ip":180,"op":301,"st":190,"bm":0},{"ddd":0,"ind":21,"ty":4,"nm":"Ã¥Â½Â¢Ã§ÂÂ¶Ã¥ÂÂ¾Ã¥Â±Â 22","sr":1,"ks":{"o":{"a":0,"k":100,"ix":11},"r":{"a":0,"k":300,"ix":10},"p":{"a":0,"k":[400,300,0],"ix":2},"a":{"a":0,"k":[1.258,2.078,0],"ix":1},"s":{"a":0,"k":[100,100,100],"ix":6}},"ao":0,"shapes":[{"ty":"gr","it":[{"d":1,"ty":"el","s":{"a":0,"k":[200,200],"ix":2},"p":{"a":0,"k":[0,0],"ix":3},"nm":"Ã¦Â¤Â­Ã¥ÂÂÃ¨Â·Â¯Ã¥Â¾Â 1","mn":"ADBE Vector Shape - Ellipse","hd":false},{"ty":"tm","s":{"a":0,"k":0,"ix":1},"e":{"a":0,"k":5,"ix":2},"o":{"a":1,"k":[{"i":{"x":[0.667],"y":[1]},"o":{"x":[0.333],"y":[0]},"t":200,"s":[0]},{"t":230,"s":[120]}],"ix":3},"m":1,"ix":2,"nm":"Ã¤Â¿Â®Ã¥ÂÂªÃ¨Â·Â¯Ã¥Â¾Â 1","mn":"ADBE Vector Filter - Trim","hd":false},{"ty":"st","c":{"a":0,"k":[0.09019607843137255,0.3843137254901961,0.8745098039215686,1],"ix":3},"o":{"a":0,"k":100,"ix":4},"w":{"a":0,"k":40,"ix":5},"lc":2,"lj":1,"ml":4,"bm":0,"nm":"Ã¦ÂÂÃ¨Â¾Â¹ 1","mn":"ADBE Vector Graphic - Stroke","hd":false},{"ty":"tr","p":{"a":0,"k":[1.258,2.078],"ix":2},"a":{"a":0,"k":[0,0],"ix":1},"s":{"a":0,"k":[100,100],"ix":3},"r":{"a":0,"k":0,"ix":6},"o":{"a":0,"k":100,"ix":7},"sk":{"a":0,"k":0,"ix":4},"sa":{"a":0,"k":0,"ix":5},"nm":"Ã¥ÂÂÃ¦ÂÂ¢"}],"nm":"Ã¦Â¤Â­Ã¥ÂÂ 1","np":4,"cix":2,"bm":0,"ix":1,"mn":"ADBE Vector Group","hd":false}],"ip":180,"op":301,"st":200,"bm":0},{"ddd":0,"ind":22,"ty":4,"nm":"Ã¥Â½Â¢Ã§ÂÂ¶Ã¥ÂÂ¾Ã¥Â±Â 21","sr":1,"ks":{"o":{"a":0,"k":100,"ix":11},"r":{"a":0,"k":280,"ix":10},"p":{"a":0,"k":[400,300,0],"ix":2},"a":{"a":0,"k":[1.258,2.078,0],"ix":1},"s":{"a":0,"k":[100,100,100],"ix":6}},"ao":0,"shapes":[{"ty":"gr","it":[{"d":1,"ty":"el","s":{"a":0,"k":[200,200],"ix":2},"p":{"a":0,"k":[0,0],"ix":3},"nm":"Ã¦Â¤Â­Ã¥ÂÂÃ¨Â·Â¯Ã¥Â¾Â 1","mn":"ADBE Vector Shape - Ellipse","hd":false},{"ty":"tm","s":{"a":0,"k":0,"ix":1},"e":{"a":0,"k":5,"ix":2},"o":{"a":1,"k":[{"i":{"x":[0.667],"y":[1]},"o":{"x":[0.333],"y":[0]},"t":210,"s":[0]},{"t":240,"s":[120]}],"ix":3},"m":1,"ix":2,"nm":"Ã¤Â¿Â®Ã¥ÂÂªÃ¨Â·Â¯Ã¥Â¾Â 1","mn":"ADBE Vector Filter - Trim","hd":false},{"ty":"st","c":{"a":0,"k":[0.09019607843137255,0.3843137254901961,0.8745098039215686,1],"ix":3},"o":{"a":0,"k":100,"ix":4},"w":{"a":0,"k":40,"ix":5},"lc":2,"lj":1,"ml":4,"bm":0,"nm":"Ã¦ÂÂÃ¨Â¾Â¹ 1","mn":"ADBE Vector Graphic - Stroke","hd":false},{"ty":"tr","p":{"a":0,"k":[1.258,2.078],"ix":2},"a":{"a":0,"k":[0,0],"ix":1},"s":{"a":0,"k":[100,100],"ix":3},"r":{"a":0,"k":0,"ix":6},"o":{"a":0,"k":100,"ix":7},"sk":{"a":0,"k":0,"ix":4},"sa":{"a":0,"k":0,"ix":5},"nm":"Ã¥ÂÂÃ¦ÂÂ¢"}],"nm":"Ã¦Â¤Â­Ã¥ÂÂ 1","np":4,"cix":2,"bm":0,"ix":1,"mn":"ADBE Vector Group","hd":false}],"ip":180,"op":301,"st":210,"bm":0},{"ddd":0,"ind":23,"ty":4,"nm":"Ã¥Â½Â¢Ã§ÂÂ¶Ã¥ÂÂ¾Ã¥Â±Â 20","sr":1,"ks":{"o":{"a":0,"k":100,"ix":11},"r":{"a":0,"k":260,"ix":10},"p":{"a":0,"k":[400,300,0],"ix":2},"a":{"a":0,"k":[1.258,2.078,0],"ix":1},"s":{"a":0,"k":[100,100,100],"ix":6}},"ao":0,"shapes":[{"ty":"gr","it":[{"d":1,"ty":"el","s":{"a":0,"k":[200,200],"ix":2},"p":{"a":0,"k":[0,0],"ix":3},"nm":"Ã¦Â¤Â­Ã¥ÂÂÃ¨Â·Â¯Ã¥Â¾Â 1","mn":"ADBE Vector Shape - Ellipse","hd":false},{"ty":"tm","s":{"a":0,"k":0,"ix":1},"e":{"a":0,"k":5,"ix":2},"o":{"a":1,"k":[{"i":{"x":[0.667],"y":[1]},"o":{"x":[0.333],"y":[0]},"t":220,"s":[0]},{"t":250,"s":[120]}],"ix":3},"m":1,"ix":2,"nm":"Ã¤Â¿Â®Ã¥ÂÂªÃ¨Â·Â¯Ã¥Â¾Â 1","mn":"ADBE Vector Filter - Trim","hd":false},{"ty":"st","c":{"a":0,"k":[0.09019607843137255,0.3843137254901961,0.8745098039215686,1],"ix":3},"o":{"a":0,"k":100,"ix":4},"w":{"a":0,"k":40,"ix":5},"lc":2,"lj":1,"ml":4,"bm":0,"nm":"Ã¦ÂÂÃ¨Â¾Â¹ 1","mn":"ADBE Vector Graphic - Stroke","hd":false},{"ty":"tr","p":{"a":0,"k":[1.258,2.078],"ix":2},"a":{"a":0,"k":[0,0],"ix":1},"s":{"a":0,"k":[100,100],"ix":3},"r":{"a":0,"k":0,"ix":6},"o":{"a":0,"k":100,"ix":7},"sk":{"a":0,"k":0,"ix":4},"sa":{"a":0,"k":0,"ix":5},"nm":"Ã¥ÂÂÃ¦ÂÂ¢"}],"nm":"Ã¦Â¤Â­Ã¥ÂÂ 1","np":4,"cix":2,"bm":0,"ix":1,"mn":"ADBE Vector Group","hd":false}],"ip":180,"op":301,"st":220,"bm":0},{"ddd":0,"ind":24,"ty":4,"nm":"Ã¥Â½Â¢Ã§ÂÂ¶Ã¥ÂÂ¾Ã¥Â±Â 19","sr":1,"ks":{"o":{"a":0,"k":100,"ix":11},"r":{"a":0,"k":240,"ix":10},"p":{"a":0,"k":[400,300,0],"ix":2},"a":{"a":0,"k":[1.258,2.078,0],"ix":1},"s":{"a":0,"k":[100,100,100],"ix":6}},"ao":0,"shapes":[{"ty":"gr","it":[{"d":1,"ty":"el","s":{"a":0,"k":[200,200],"ix":2},"p":{"a":0,"k":[0,0],"ix":3},"nm":"Ã¦Â¤Â­Ã¥ÂÂÃ¨Â·Â¯Ã¥Â¾Â 1","mn":"ADBE Vector Shape - Ellipse","hd":false},{"ty":"tm","s":{"a":0,"k":0,"ix":1},"e":{"a":0,"k":5,"ix":2},"o":{"a":1,"k":[{"i":{"x":[0.667],"y":[1]},"o":{"x":[0.333],"y":[0]},"t":230,"s":[0]},{"t":260,"s":[120]}],"ix":3},"m":1,"ix":2,"nm":"Ã¤Â¿Â®Ã¥ÂÂªÃ¨Â·Â¯Ã¥Â¾Â 1","mn":"ADBE Vector Filter - Trim","hd":false},{"ty":"st","c":{"a":0,"k":[0.09019607843137255,0.3843137254901961,0.8745098039215686,1],"ix":3},"o":{"a":0,"k":100,"ix":4},"w":{"a":0,"k":40,"ix":5},"lc":2,"lj":1,"ml":4,"bm":0,"nm":"Ã¦ÂÂÃ¨Â¾Â¹ 1","mn":"ADBE Vector Graphic - Stroke","hd":false},{"ty":"tr","p":{"a":0,"k":[1.258,2.078],"ix":2},"a":{"a":0,"k":[0,0],"ix":1},"s":{"a":0,"k":[100,100],"ix":3},"r":{"a":0,"k":0,"ix":6},"o":{"a":0,"k":100,"ix":7},"sk":{"a":0,"k":0,"ix":4},"sa":{"a":0,"k":0,"ix":5},"nm":"Ã¥ÂÂÃ¦ÂÂ¢"}],"nm":"Ã¦Â¤Â­Ã¥ÂÂ 1","np":4,"cix":2,"bm":0,"ix":1,"mn":"ADBE Vector Group","hd":false}],"ip":180,"op":301,"st":230,"bm":0}]}],"layers":[{"ddd":0,"ind":1,"ty":0,"nm":"Ã©Â¢ÂÃ¥ÂÂÃ¦ÂÂ 1","refId":"comp_0","sr":1,"ks":{"o":{"a":0,"k":100,"ix":11},"r":{"a":0,"k":0,"ix":10},"p":{"a":0,"k":[400,300,0],"ix":2},"a":{"a":0,"k":[400,300,0],"ix":1},"s":{"a":0,"k":[100,100,100],"ix":6}},"ao":0,"ef":[{"ty":5,"nm":"Ã§Â®ÂÃ¥ÂÂÃ©ÂÂ»Ã¥Â¡ÂÃ¥Â·Â¥Ã¥Â
Â·","np":4,"mn":"ADBE Simple Choker","ix":1,"en":1,"ef":[{"ty":7,"nm":"Ã¨Â§ÂÃ¥ÂÂ¾","mn":"ADBE Simple Choker-0001","ix":1,"v":{"a":0,"k":1,"ix":1}},{"ty":0,"nm":"Ã©ÂÂ»Ã¥Â¡ÂÃ©ÂÂ®Ã§Â½Â©","mn":"ADBE Simple Choker-0002","ix":2,"v":{"a":0,"k":14,"ix":2}}]},{"ty":5,"nm":"Ã¦Â¢Â¯Ã¥ÂºÂ¦Ã¦Â¸ÂÃ¥ÂÂ","np":10,"mn":"ADBE Ramp","ix":2,"en":1,"ef":[{"ty":3,"nm":"Ã¦Â¸ÂÃ¥ÂÂÃ¨ÂµÂ·Ã§ÂÂ¹","mn":"ADBE Ramp-0001","ix":1,"v":{"a":0,"k":[400,0],"ix":1}},{"ty":2,"nm":"Ã¨ÂµÂ·Ã¥Â§ÂÃ©Â¢ÂÃ¨ÂÂ²","mn":"ADBE Ramp-0002","ix":2,"v":{"a":0,"k":[0.263066768646,1,0.867243647575,1],"ix":2}},{"ty":3,"nm":"Ã¦Â¸ÂÃ¥ÂÂÃ§Â»ÂÃ§ÂÂ¹","mn":"ADBE Ramp-0003","ix":3,"v":{"a":0,"k":[400,600],"ix":3}},{"ty":2,"nm":"Ã§Â»ÂÃ¦ÂÂÃ©Â¢ÂÃ¨ÂÂ²","mn":"ADBE Ramp-0004","ix":4,"v":{"a":0,"k":[0.354580283165,1,0.820018112659,1],"ix":4}},{"ty":7,"nm":"Ã¦Â¸ÂÃ¥ÂÂÃ¥Â½Â¢Ã§ÂÂ¶","mn":"ADBE Ramp-0005","ix":5,"v":{"a":0,"k":1,"ix":5}},{"ty":0,"nm":"Ã¦Â¸ÂÃ¥ÂÂÃ¦ÂÂ£Ã¥Â°Â","mn":"ADBE Ramp-0006","ix":6,"v":{"a":0,"k":0,"ix":6}},{"ty":0,"nm":"Ã¤Â¸ÂÃ¥ÂÂÃ¥Â§ÂÃ¥ÂÂ¾Ã¥ÂÂÃ¦Â·Â·Ã¥ÂÂ","mn":"ADBE Ramp-0007","ix":7,"v":{"a":0,"k":0,"ix":7}},{"ty":6,"nm":"","mn":"ADBE Ramp-0008","ix":8,"v":0}]}],"w":800,"h":600,"ip":0,"op":300,"st":0,"bm":0},{"ddd":0,"ind":2,"ty":0,"nm":"Ã©Â¢ÂÃ¥ÂÂÃ¦ÂÂ 1","refId":"comp_0","sr":1,"ks":{"o":{"a":0,"k":56,"ix":11},"r":{"a":0,"k":0,"ix":10},"p":{"a":0,"k":[400,320,0],"ix":2},"a":{"a":0,"k":[400,300,0],"ix":1},"s":{"a":0,"k":[100,100,100],"ix":6}},"ao":0,"ef":[{"ty":5,"nm":"Ã§Â®ÂÃ¥ÂÂÃ©ÂÂ»Ã¥Â¡ÂÃ¥Â·Â¥Ã¥Â
Â·","np":4,"mn":"ADBE Simple Choker","ix":1,"en":1,"ef":[{"ty":7,"nm":"Ã¨Â§ÂÃ¥ÂÂ¾","mn":"ADBE Simple Choker-0001","ix":1,"v":{"a":0,"k":1,"ix":1}},{"ty":0,"nm":"Ã©ÂÂ»Ã¥Â¡ÂÃ©ÂÂ®Ã§Â½Â©","mn":"ADBE Simple Choker-0002","ix":2,"v":{"a":0,"k":14,"ix":2}}]},{"ty":5,"nm":"Ã¦Â¢Â¯Ã¥ÂºÂ¦Ã¦Â¸ÂÃ¥ÂÂ","np":10,"mn":"ADBE Ramp","ix":2,"en":1,"ef":[{"ty":3,"nm":"Ã¦Â¸ÂÃ¥ÂÂÃ¨ÂµÂ·Ã§ÂÂ¹","mn":"ADBE Ramp-0001","ix":1,"v":{"a":0,"k":[400,0],"ix":1}},{"ty":2,"nm":"Ã¨ÂµÂ·Ã¥Â§ÂÃ©Â¢ÂÃ¨ÂÂ²","mn":"ADBE Ramp-0002","ix":2,"v":{"a":0,"k":[0.263066768646,1,0.867243647575,1],"ix":2}},{"ty":3,"nm":"Ã¦Â¸ÂÃ¥ÂÂÃ§Â»ÂÃ§ÂÂ¹","mn":"ADBE Ramp-0003","ix":3,"v":{"a":0,"k":[400,600],"ix":3}},{"ty":2,"nm":"Ã§Â»ÂÃ¦ÂÂÃ©Â¢ÂÃ¨ÂÂ²","mn":"ADBE Ramp-0004","ix":4,"v":{"a":0,"k":[0.354580283165,1,0.820018112659,1],"ix":4}},{"ty":7,"nm":"Ã¦Â¸ÂÃ¥ÂÂÃ¥Â½Â¢Ã§ÂÂ¶","mn":"ADBE Ramp-0005","ix":5,"v":{"a":0,"k":1,"ix":5}},{"ty":0,"nm":"Ã¦Â¸ÂÃ¥ÂÂÃ¦ÂÂ£Ã¥Â°Â","mn":"ADBE Ramp-0006","ix":6,"v":{"a":0,"k":0,"ix":6}},{"ty":0,"nm":"Ã¤Â¸ÂÃ¥ÂÂÃ¥Â§ÂÃ¥ÂÂ¾Ã¥ÂÂÃ¦Â·Â·Ã¥ÂÂ","mn":"ADBE Ramp-0007","ix":7,"v":{"a":0,"k":0,"ix":7}},{"ty":6,"nm":"","mn":"ADBE Ramp-0008","ix":8,"v":0}]},{"ty":29,"nm":"Ã©Â«ÂÃ¦ÂÂ¯Ã¦Â¨Â¡Ã§Â³Â","np":5,"mn":"ADBE Gaussian Blur 2","ix":3,"en":1,"ef":[{"ty":0,"nm":"Ã¦Â¨Â¡Ã§Â³ÂÃ¥ÂºÂ¦","mn":"ADBE Gaussian Blur 2-0001","ix":1,"v":{"a":0,"k":41.3,"ix":1}},{"ty":7,"nm":"Ã¦Â¨Â¡Ã§Â³ÂÃ¦ÂÂ¹Ã¥ÂÂ","mn":"ADBE Gaussian Blur 2-0002","ix":2,"v":{"a":0,"k":1,"ix":2}},{"ty":7,"nm":"Ã©ÂÂÃ¥Â¤ÂÃ¨Â¾Â¹Ã§Â¼ÂÃ¥ÂÂÃ§Â´Â ","mn":"ADBE Gaussian Blur 2-0003","ix":3,"v":{"a":0,"k":0,"ix":3}}]}],"w":800,"h":600,"ip":0,"op":300,"st":0,"bm":0}],"markers":[]}

In [ ]:
!npm install localtunnel


added 22 packages, and audited 23 packages in 2s

3 packages are looking for funding
  run `npm fund` for details

2 moderate severity vulnerabilities

To address all issues, run:
  npm audit fix

Run `npm audit` for details.


In [ ]:
!streamlit run /content/app.py &>/content/logs.txt &

In [ ]:
!curl https://loca.lt/mytunnelpassword

35.245.231.108

In [ ]:
!npx localtunnel --port 8501

your url is: https://solid-pets-post.loca.lt
